In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import plotly.figure_factory as ff
import nltk
nltk.download('stopwords')

In [ ]:
#set display option for rows & columns

pd.set_option( 'display.max_rows', None)
pd.set_option('display.max_columns', None )

In [ ]:
train_df = pd.read_csv('/content/train.csv')
print("Shape of train data : ",train_df.shape)

In [ ]:
# checkout null value
train_df.isnull().sum()
# dropping the missing values
train_df.dropna(axis=0, how='any', inplace=True)
#resetting index
train_df.reset_index(inplace = True, drop = True)

In [ ]:
print("Shape of train data : ",train_df.shape)

In [ ]:
train_df.head(3)

In [ ]:
temp = train_df.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Pastel1')

In [ ]:
#remove specical charater

def removespecifchar(df):
  special_characters = "`"
  for k in range(len(df)):
    if(special_characters in df[k]):
      df[k]=df[k].replace("`","'")

#Train dataset
removespecifchar(train_df['text'])
removespecifchar(train_df['selected_text'])

In [ ]:
# define contrations

contrations = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


In [ ]:
def decontracted(text):
  for word in text.split():
    if word.lower() in contrations:
        text = text.replace(word, contrations[word.lower()])
    return text

In [ ]:
#Train dataset
train_df['text'] = train_df['text'].apply(lambda text: decontracted(text))
train_df['selected_text'] = train_df['selected_text'].apply(lambda text: decontracted(text))

In [ ]:
def clean_text(text):
    '''
        1) Making text lowercase
        2) Remove text in square brackets
        3) Remove links
        4) Remove punctuation,
           and remove words containing numbers.

    '''

    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
#Train dataset
train_df['text'] = train_df['text'].apply(lambda x:clean_text(x))
train_df['selected_text'] = train_df['selected_text'].apply(lambda x:clean_text(x))

In [ ]:
#Empty strings row check and removing
train_df=train_df[train_df["selected_text"]!=""]
train_df=train_df[train_df["text"]!=""]
train_df.reset_index(inplace = True, drop = True)

In [ ]:
print(train_df.shape)

In [ ]:
#removing Stopwords

from nltk.corpus import stopwords, wordnet
def remove_Stopwords(text):

    #to get the english stopwords
    stopW = stopwords.words('english')
    return " ".join([i for i in text.split() if i not in stopW])

In [ ]:
train_df['clean_text'] = train_df['text'].apply(lambda x:remove_Stopwords(x))

In [ ]:
train_df.head(3)

In [ ]:
train_df = train_df[['textID','text','selected_text','sentiment']]

In [ ]:
train_df.to_csv('/content/train_preprocess.csv',index=False)